# Introduction to Convolutional Neural Networks


** Ecole Centrale Nantes **

** Diana Mateus **


** Participants : **
Diane Chounlamountry
Orane Vernet


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

### 0. Loading the dataset
Start by runing the following lines to load and visualize the data.

In [8]:
# UNCOMMENT IF USING COLAB
# from google.colab import drive
# drive.mount('/content/drive')
IMDIR = '//data-pfe/overnet2022/EI2/APSTA/TP4/'

In [9]:
def load_dataset(IMDIR):
    train_dataset = h5py.File(IMDIR+'dataset/train_catvnoncat.h5', "r")
    train_x = np.array(train_dataset["train_set_x"][:])
    train_y = np.array(train_dataset["train_set_y"][:])
    test_dataset = h5py.File(IMDIR+'dataset/test_catvnoncat.h5', "r")
    test_x = np.array(test_dataset["test_set_x"][:])
    test_y = np.array(test_dataset["test_set_y"][:])
    classes = np.array(test_dataset["list_classes"][:])

    train_y = train_y.reshape((1, train_y.shape[0]))
    test_y = test_y.reshape((1, test_y.shape[0]))

    return train_x, train_y, test_x, test_y, classes

train_x, train_y, test_x, test_y, classes=load_dataset(IMDIR)

#### Visualize data

In [1]:
# run several times to visualize different data points
# the title shows the ground truth class labels (0=no cat , 1 = cat)
index = np.random.randint(low=0,high=train_y.shape[1])
plt.imshow(train_x[index])
plt.title("Image "+str(index)+" label "+str(train_y[0,index]))
plt.show()
print ("Train X shape: " + str(train_x.shape))
print ("We have "+str(train_x.shape[0]),
       "images of dimensionality "
       + str(train_x.shape[1])+ "x"
       + str(train_x.shape[2])+ "x"
       + str(train_x.shape[3]))

NameError: name 'np' is not defined

### 1. CNNs with Keras and Tensorflow

Adapt the example in this website https://keras.io/examples/vision/mnist_convnet/ to our problem. To this end:
- change the number of classes and the input size
- remove the expand_dims(x_train, -1): it is not necessary to expand the dimensions since our input has already three channels
- you may need to transpose the labels vector
- change the categorical cross-entropy to the binary cross entropy given that our problem is binary classification.
- also change the softmax to sigmoid, the more appropriate activation function for binary data

We can choose a single neuron output passed through sigmoid, and then set a threshold to choose the class, or use two neuron output and then perform a softmax.

**2.2** Compute the train and test loss and accuracy after the model has been trained.  What model parameters does the ``fit`` function retain?

**2.3** How many parameters does the network have, explain  the exact number .

**2.4** Display and discuss the ROC curve of at least 3 different CNN configurations  


In [13]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
# the data, split between train and test sets
x_train, y_train, x_test, y_test, classes=load_dataset(IMDIR)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# convert class vectors to binary class matrices
y_train = y_train.T
y_test = y_test.T




In [21]:
#build the model

model = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(2, activation="sigmoid"),
    ]
)

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 12544)             0         
                                                                 
 dropout_3 (Dropout)         (None, 12544)            

In [24]:
#comiple and fit
batch_size = 128
epochs = 15

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15


ValueError: in user code:

    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "D:\Appli\Anaconda\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "D:\Appli\Anaconda\lib\site-packages\keras\l

In [ ]:
#evaluate
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## 2 Custom training loop 
Replace the fit function by your own tensorflow  implementation

- Instantiate one of keras.optimizers to train the model.

- Instantiate a loss from keras.losses

- Define the metrics (from keras.metrics)

- Use `tf.data.Dataset.from_tensor_slices` to create an iterable dataset from a numpy arrays. Do this for the training and test datasets.

- Change the model (optional, after the training loop runs to optimize the performance)

- Program a loop over a fixed number of epochs,
    * For each epoch iterating over the batches
    * Within a `GradientTape()` scope,
      - do a forward pass on the model for the current batch (call the model on the batch data)
      - Compute the loss
      - Compute the gradients of the loss w.r.t parameters
      - Call the optimimzer to update the weights with computed the gradients
    * At the end of each epoch compute the validation metrics


Look at https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough specifically at the TRAINING LOOP SECTION
for a recent documentation on custom training.


ADITIONAL BONUS
- Early stopping
- Tensorboard
- CAM/GradCAM